In [11]:
!py -m pip install langgraph-checkpoint-sqlite

# !py -m pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-openai langchain-chroma bs4

  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 2.0.9
    Uninstalling langgraph-checkpoint-2.0.9:
      Successfully uninstalled langgraph-checkpoint-2.0.9



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
import pandas as pd
import os
import shutil
import numpy as np
from uuid import uuid4
from langchain_community.document_loaders import PyPDFLoader, UnstructuredWordDocumentLoader
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.docstore.document import Document
from typing import List, Tuple
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore
from langgraph.store.base import BaseStore
from langchain_core.messages import SystemMessage, HumanMessage
import re
from typing import Annotated, TypedDict
import operator
from langchain_core.documents import Document

In [3]:
OPENAI_DEPLOYMENT_ENDPOINT = "https://advancedanalyticsopenaikey.openai.azure.com/"
OPENAI_DEPLOYMENT_ENDPOINT_embed = "https://pkl-aa-dev-aiservices.openai.azure.com/" 
OPENAI_API_KEY = "FqFd4DBx1W97MSVjcZvdQsmQlhI80hXjl48iWYmZ4W3NutUlWvf0JQQJ99BDACYeBjFXJ3w3AAABACOGl3xo" 
OPENAI_API_VERSION = "2024-12-01-preview"
OPENAI_API_KEY_EMBEDDINGS = "AXEC3y1jC9ZNGCBB12NZwrpBSzScq1esexgvCXiqw7PaHE04vSMbJQQJ99BDACYeBjFXJ3w3AAABACOG4CMN" 
OPENAI_DEPLOYMENT_NAME = "gpt-4o"
OPENAI_MODEL_NAME="gpt-4o"
embedding_api_version = "2024-02-01"

# OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
# CHROMA_PATH = "c:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\SERFF\\vector_db\\testing_db"
DB_PATH  = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\vector_db"
DATA_FOLDER = "C:\\Users\\SujaySunilNagvekar\\VM\\GEN AI\\KM\\Documents"

### Cohere API key 
# cohere_API_key = "wHyiTViP32Y3Q8Qwhjmd4QGNCkYNpxqtsemtSri3"
# co = cohere.Client(cohere_API_key)

In [4]:

llm = AzureChatOpenAI(
                        temperature=0.7,
                        deployment_name=OPENAI_DEPLOYMENT_NAME,
                        model_name=OPENAI_MODEL_NAME,
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
                        openai_api_version=OPENAI_API_VERSION,
                        openai_api_key=OPENAI_API_KEY            
                    )

embeddings = AzureOpenAIEmbeddings(
                        deployment="text-embedding-3-small",
                        model="text-embedding-3-small",
                        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT_embed,
                        openai_api_version=embedding_api_version,
                        openai_api_key=OPENAI_API_KEY_EMBEDDINGS)


In [1]:
import pandas as pd
from tqdm import tqdm
from langchain_core.messages import SystemMessage, HumanMessage
from pydantic import BaseModel, Field
from typing import Optional
import random
from time import time
import unicodedata

import pandas as pd
import random
import uuid
from faker import Faker
from datetime import datetime

# Load claims file
claims_df = pd.read_csv("C://Users//SujaySunilNagvekar//VM//GEN AI//KM//new_claims_data.csv")
fake = Faker()
log_rows = []

for _, row in claims_df.iterrows():
    try:
        # Extract directly from claims.csv without randomization
        claim_number = row["Claim Number"]
        policy_number = row["Policy Number"]
        loss_date = row["Date of Loss"]
        reported_date = row["Date of Reporting Loss"]
        claimant = row.get("Claimant Name", fake.name())
        claim_status = row.get("Claim Status", "Unknown")
        litigation = row.get("Litigation", "Unknown")
        loss_cause = row.get("Loss cause", "Unknown")
        loss_state = row.get("Loss Location State", "Unknown")
        medical_bill = row.get("Medical bill", 0)
        subro_status = row.get("Subro Opportunity ", "Unknown")

        # Add mock log layer on top
        log_rows.append({
            "Timestamp": fake.date_time_between(start_date='-2y', end_date='now'),
            "Log Level": random.choice(["INFO", "DEBUG", "WARN", "ERROR", "FATAL"]),
            "Thread ID": uuid.uuid4().hex[:8],
            "Class/Module": random.choice(["claims.service", "payment.processor", "document.manager"]),
            "Message": fake.sentence(),
            "UserID / Adjuster ID": f"A{random.randint(1000, 9999)}",

            # 🚫 DO NOT RANDOMIZE
            "Claim Number (ClaimID)": claim_number,
            "Policy Number": policy_number,
            "ClaimID": claim_number,
            "Loss Date": loss_date,
            "Reported Date": reported_date,
            "Claimant Name / Contact Info": f"{claimant}, {fake.phone_number()}",
            "Claim Status": claim_status,
            "Litigation Indicator": litigation,
            "Loss Type": loss_cause,
            "Jurisdiction": loss_state,
            "Payment Amounts": medical_bill,
            "Subrogation Status": subro_status,

            # ✅ Add-on log fields
            "Transaction ID": uuid.uuid4().hex[:10],
            "Event Type": random.choice(["Claim Initiated", "Payment Processed", "Document Uploaded"]),
            "Adjuster Assigned": f"Adjuster {fake.last_name()}",
            "Exposure ID": f"E{random.randint(10000,99999)}",
            "Reserve Amounts": round(random.uniform(1000, 50000), 2),
            "Recovery Amounts": round(random.uniform(0, 20000), 2),
            "Activities Logged": random.randint(1, 10),
            "Documents Generated": random.randint(0, 5),
            "Attachments": random.randint(0, 3),
            "External System Status": random.choice(["Success", "Failure", "Pending"]),
            "Audit Trail Entries": random.randint(1, 10),
            "Request Timestamp": fake.date_time_between(start_date='-2y', end_date='now'),
            "Request Payload": fake.text(max_nb_chars=100),
            "Response Code": random.choice([200, 201, 400, 500]),
            "Response Time": round(random.uniform(0.1, 2.5), 3),
            "Error Stacktrace": fake.sentence(nb_words=6) if random.random() < 0.2 else "",
            "Correlation ID / Trace ID": uuid.uuid4().hex,
            "External System Name": random.choice(["Guidewire", "Duck Creek", "Salesforce"]),
        })

    except Exception as e:
        print(f"❌ Error processing row: {e}")

# Save logs
logs_df = pd.DataFrame(log_rows)
logs_df.to_csv("C://Users//SujaySunilNagvekar//VM//GEN AI//KM//logs.csv", index=False, encoding="utf-8")
print("✅ logs.csv generated with claim-aligned data.")



OSError: [Errno 28] No space left on device